In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(9,4.5))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
import qubic.demodulation_lib as dl
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
# import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
from scipy.signal import medfilt
import matplotlib.pyplot as plt
import glob
import string
# import pickle
from importlib import reload
from scipy.interpolate import interp1d
import healpy as hp
import os

import DataHandling as DH
import pymap3d

from qubicpack.utilities import Qubic_DataDir
import qubic
from qubic import selfcal_lib as scal
from qubic import fibtools as ft

from qubic import progress_bar
def identify_scans(thk, az, el, tt=None, median_size=101, thr_speedmin=0.1, doplot=False, plotrange=[0,1000]):
    """
    This function identifies and assign numbers the various regions of a back-and-forth scanning using the housepkeeping time, az, el
        - a numbering for each back & forth scan
        - a region to remove at the end of each scan (bad data due to FLL reset, slowingg down of the moiunt, possibly HWP rotation
        - is the scan back or forth ?
    It optionnaly iinterpolate this information to the TOD sampling iif provided.
    Parameters
    ----------
    input
    thk : np.array()
            time samples (seconds) for az and el at the housekeeeping sampling rate
    az : np.array()
            azimuth in degrees at the housekeeping sampling rate
    el : np.array()
            elevation in degrees at the housekeeping sampling rate
    tt : Optional : np.array()
            None buy default, if not None:
            time samples (seconds) at the TOD sampling rate
            Then. the output will also containe az,el and scantype interpolated at TOD sampling rate
    thr_speedmin : Optional : float
            Threshold for angular velocity to be considered as slow
    doplot : [Optional] : Boolean
            If True displays some useeful plot
    output : 
    scantype_hk: np.array(int)
            type of scan for each sample at housekeeping sampling rate:
            * 0 = speed to slow - Bad data
            * n = scanning towards positive azimuth
            * -n = scanning towards negative azimuth
            where n is the scan number (starting at 1)
    azt : [optional] np.array()
            azimuth (degrees) at the TOD sampling rate
    elt : [optional] np.array()
            elevation (degrees) at the TOD sampling rate
    scantype : [optiona] np.array()
            same as scantype_hk, but interpolated at TOD sampling rate
    """
    
    ### Sampling for HK data
    timesample = np.median(thk[1:]-thk[:-1])
    ### angular velocity 
    medaz_dt = medfilt(np.gradient(az, timesample), median_size)
    ### Identify regions of change
    # Low velocity -> Bad
    c0 = np.abs(medaz_dt) < thr_speedmin
    # Positive velicity => Good
    cpos = (~c0) * (medaz_dt > 0)
    # Negative velocity => Good
    cneg = (~c0) * (medaz_dt < 0)
    
    ### Scan identification at HK sampling
    scantype_hk = np.zeros(len(thk), dtype='int')-10
    scantype_hk[c0] =0
    scantype_hk[cpos] = 1
    scantype_hk[cneg] = -1
    # check that we have them all
    count_them = np.sum(scantype_hk==0) + np.sum(scantype_hk==-1) + np.sum(scantype_hk==1)
    if count_them != len(scantype_hk):
        print('Identify_scans: Bad Scan counting at HK sampling level - Error')
        stop
    
    ### Now give a number to each back and forth scan
    num = 0
    previous = 0
    for i in range(len(scantype_hk)):
        if scantype_hk[i] == 0:
            previous = 0
        else:
            if (previous == 0) & (scantype_hk[i] > 0):
                # we have a change
                num += 1
            previous = 1
            scantype_hk[i] *= num
            
    dead_time = np.sum(c0) / len(thk)
    
    if doplot:
        ### Some plotting
        plt.subplot(2,2,1)
        plt.title('Angular Velocity Vs. Azimuth - Dead time = {0:4.1f}%'.format(dead_time*100))
        plt.plot(az, medaz_dt)
        plt.plot(az[c0], medaz_dt[c0], 'ro', label='Slow speed')
        plt.plot(az[cpos], medaz_dt[cpos], '.', label='Scan +')
        plt.plot(az[cneg], medaz_dt[cneg], '.', label='Scan -')
        plt.xlabel('Azimuth [deg]')
        plt.ylabel('Ang. velocity [deg/s]')
        plt.legend(loc='upper left')
        plt.subplot(2,2,2)
        plt.title('Angular Velocity Vs. time - Dead time = {0:4.1f}%'.format(dead_time*100))
        plt.plot(thk, medaz_dt)
        plt.plot(thk[c0], medaz_dt[c0], 'ro', label='speed=0')
        plt.plot(thk[cpos], medaz_dt[cpos], '.', label='Scan +')
        plt.plot(thk[cneg], medaz_dt[cneg], '.', label='Scan -')
        plt.legend(loc='upper left')
        plt.xlabel('Time [s]')
        plt.ylabel('Ang. velocity [deg/s]')
        plt.xlim(plotrange[0],plotrange[1])
        plt.subplot(2,3,4)
        plt.title('Azimuth Vs. time - Dead time = {0:4.1f}%'.format(dead_time*100))
        plt.plot(thk, az)
        plt.plot(thk[c0], az[c0], 'ro', label='speed=0')
        plt.plot(thk[cpos], az[cpos], '.', label='Scan +')
        plt.plot(thk[cneg], az[cneg], '.', label='Scan -')
        plt.legend(loc='upper left')
        plt.xlabel('Time [s]')
        plt.ylabel('Azimuth [deg]')
        plt.xlim(plotrange[0],plotrange[1])
        plt.subplot(2,3,5)
        plt.title('Elevation Vs. time - Dead time = {0:4.1f}%'.format(dead_time*100))
        plt.plot(thk, el)
        plt.plot(thk[c0], el[c0], 'ro', label='speed=0')
        plt.plot(thk[cpos], el[cpos], '.', label='Scan +')
        plt.plot(thk[cneg], el[cneg], '.', label='Scan -')
        plt.legend(loc='lower left')
        plt.xlabel('Time [s]')
        plt.ylabel('Elevtion [deg]')
        plt.xlim(plotrange[0],plotrange[1])
        elvals = el[(thk > plotrange[0]) & (thk < plotrange[1])]
        deltael = np.max(elvals) - np.min(elvals)
        plt.ylim(np.min(elvals) - deltael/5, np.max(elvals) + deltael/5)
        
        allnums = np.unique(np.abs(scantype_hk))
        for n in allnums[allnums > 0]:
            ok = np.abs(scantype_hk) == n
            xx = np.mean(thk[ok]) 
            yy = np.mean(el[ok])
            if (xx > plotrange[0])  & (xx < plotrange[1]):
                plt.text(xx, yy+deltael/20, str(n))
                
        plt.subplot(2,3,6)
        plt.title('Elevation Vs. time - Dead time = {0:4.1f}%'.format(dead_time*100))
        thecol = (np.arange(len(allnums))*256/(len(allnums)-1)).astype(int)
        for i in range(len(allnums)):
            ok = np.abs(scantype_hk) == allnums[i]
            plt.plot(az[ok], el[ok], color=plt.get_cmap(plt.rcParams['image.cmap'])(thecol[i]))
        plt.ylim(np.min(el), np.max(el))
        plt.xlabel('Azimuth [deg]')
        plt.ylabel('Elevtion [deg]')
        plt.scatter(-allnums*0, -allnums*0-10,c=allnums)
        aa=plt.colorbar()
        aa.ax.set_ylabel('Scan number')        
        #plt.tight_layout()
    if tt is not None:
        ### We propagate these at TOD sampling rate  (this is an "step interpolation": we do not want intermediatee values")
        scantype = interp1d(thk, scantype_hk, kind='previous', fill_value='extrapolate')(tt)
        scantype = scantype.astype(int)
        count_them = np.sum(scantype==0) + np.sum(scantype<=-1) + np.sum(scantype>=1)
        if count_them != len(scantype):
            print('Bad Scan counting at data sampling level - Error')
            stop
        ### Interpolate azimuth and elevation to TOD sampling
        azt = np.interp(tt, thk, az)
        elt = np.interp(tt, thk, el)
        ### Return evereything
        return scantype_hk, azt, elt, scantype
    else:
        ### Return scantype at HK sampling only
        return scantype_hk
def get_chunks(mytod, scantype, value):
    ### returns chunks corresponding to a given value
    current_chunk = []
    chunk_idx = []
    inchunk = 0
    chunknum = 0
    for i in range(len(scantype)):
        if scantype[i]==value:
            inchunk = 1
            current_chunk.append(i)
        else:
            if inchunk == 1:
                chunknum += 1
                chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
                current_chunk = []
                inchunk = 0
    if inchunk == 1:
        chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
    return chunk_idx
def linear_rescale_chunks(mytod, chunks, sz=1000):
    for i in range(len(chunks)):
        thechunk = chunks[i]
        chunklen = thechunk[1] - thechunk[0]+1
        if thechunk[0] == 0:
            # this is the starting index => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[1]+1: thechunk[1]+sz]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        elif thechunk[1]==(len(mytod)-1):
            # this is the last one => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        else:
            left = np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1])
            right = np.median(mytod[thechunk[1]+1: thechunk[1]+sz])
            vals = left + np.linspace(0,1, chunklen)*(right-left)
            mytod[thechunk[0]:thechunk[1]+1] -= np.median(mytod[thechunk[0]:thechunk[1]+1]) - vals
            
    return mytod

def decorel_azel(mytod, azt, elt, scantype, doplot=True, n_el=20, degree=3):
    ### Profiling in Azimuth and elevation
    el_lims = np.linspace(np.min(el)-0.0001, np.max(el)+0.0001, n_el+1)
    el_av = 0.5 * (el_lims[1:] + el_lims[:-1])
    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    minaz = np.min(azt[okall])
    maxaz = np.max(azt[okall])
    xaz = np.linspace(minaz, maxaz, 100)
    if doplot:
        plt.figure()
        plt.xlabel('Azimuth [deg]')
        plt.ylabel('Mode of TOD')
    
    coefficients = np.zeros((2, n_el, degree+1))
    for i in range(len(oks)):
        if doplot: plt.subplot(1,2,i+1)
        for j in range(n_el):
            ok = oks[i] & (elt >= el_lims[j]) & (elt < el_lims[j+1])
            xc, yc, dx, dy, _ = ft.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=50, mode=True, dispersion=True, plot=False)
            z = np.polyfit(xc, yc, degree, w=1./dy)
            p = np.poly1d(z)
            coefficients[i,j,:] = z
            if doplot:
                pl = plt.errorbar(xc, yc, yerr=dy, xerr=dx, fmt='o')
                plt.plot(xaz, p(xaz), color=pl[0].get_color(), label = oks_names[i] + ' - El = {0:5.1f}'.format(np.mean(elt[ok])))
    if doplot: plt.legend()
    ### Now interpolate this to remove it to the data
    nscans = np.max(np.abs(scantype))
    for i in range(1, nscans+1):
        okp = scantype == i
        okn = scantype == (-i)
        for ok in [okp, okn]:
            the_el = np.median(elt[ok])
            myp = np.poly1d([np.interp(the_el, el_av, coefficients[0,:,i]) for i in np.arange(degree+1)])
            mytod[ok] -= myp(azt[ok])
    ### And interpolate for scantype==0 regions
    bad_chunks = get_chunks(mytod, scantype, 0)
    mytod = linear_rescale_chunks(mytod, bad_chunks, sz=100)
    return mytod    
def hf_noise_estimate(tt, dd):
    sh = np.shape(dd)
    if len(sh) == 1:
        dd = np.reshape(dd, (1, len(dd)))
        ndet = 1
    else:
        ndet = sh[0]
    estimate = np.zeros(ndet)
    for i in range(ndet):
        spectrum_f, freq_f = ft.power_spectrum(tt, dd[i, :], rebin=True)
        mean_level = np.mean(spectrum_f[np.abs(freq_f) > (np.max(freq_f) / 2)])
        samplefreq = 1. / (tt[1] - tt[0])
        estimate[i] = (np.sqrt(mean_level * samplefreq / 2))

    return estimate

def remove_noise(tt, tod):
    
    hf_noise = hf_noise_estimate(tt, tod) / np.sqrt(2)
    var_diff = tod ** 2 - hf_noise ** 2
    tod = np.sqrt(np.abs(var_diff)) * np.sign(var_diff)
    return tod
    
def remove_offset_scan(mytod, scantype, method='meancut', apply_to_bad = True):
    ### We remove offsets for each good scan but we also need to remove a coomparable offset for the scantype==0 reggiions in order to keep coninuity 
    ### This si donee by apply_to_bad=True
    
    indices = np.arange(len(mytod))
    last_index = 0
    myoffsetn = 0
    myoffsetp = 0
    donefirst = 0
    
    nscans = np.max(np.abs(scantype))
    for n in range(1, nscans+1):
        # scan +
        ok = scantype == n
        if method == 'meancut':
            myoffsetp, _ = ft.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetp = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetp = get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetp        
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetn = myoffsetp ### deal with first region
            vals_offsets = myoffsetn + np.linspace(0,1, first_index-last_index-1)*(myoffsetp-myoffsetn)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
        
        
        # scan -
        ok = scantype == (-n)
        if method == 'meancut':
            myoffsetn, _ = ft.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetn = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetn = get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetn
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetp = myoffsetn ### deal with first region
            vals_offsets = myoffsetp + np.linspace(0,1, first_index-last_index-1)*(myoffsetn-myoffsetp)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
    
    return mytod

def decorel_azimuth(mytod, azt, scantype, doplot=True):
    ### Profiling in Azimuth
    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    polys = []
    if doplot:
        plt.figure(figsize=(15, 5))
    for i in range(len(oks)):
        ok = oks[i]
        minaz = np.min(azt[ok])
        maxaz = np.max(azt[ok])
        xc, yc, dx, dy, _ = ft.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=25, mode=True, dispersion=True, plot=False)
        z = np.polyfit(xc, yc, 2, w=1./dy)
        p = np.poly1d(z)
        polys.append(p)
        xaz = np.linspace(minaz, maxaz, 100)
        if doplot:
            pl = plt.errorbar(xc, yc, xerr=dx, fmt='o')
            plt.plot(xaz, p(xaz), label=oks_names[i], color=pl[0].get_color())
    if doplot:
        plt.xlabel('Azimuth [deg]')
        plt.ylabel('Mode of TOD')
        plt.legend()
    ### Removing the azimuthal effect
    ok = scantype >= 0
    mytod[ok] -= polys[0](azt[ok])
    ok = scantype < 0
    mytod[ok] -= polys[1](azt[ok])
    
    return mytod
def display_healpix_map(maps, add_moon_traj=None, savepdf=None):
    
    if add_moon_traj is not None:
        th, phi = add_moon_traj
    
    figure(figsize=(30, 30))
    #k=1
    bar=progress_bar(maps.shape[0], 'Display healpix maps')
    
    x=np.linspace(-0.0504, -0.0024, 17)
    y=np.linspace(-0.0024, -0.0504, 17)

    X, Y = np.meshgrid(x, y)
    
    allTES=np.arange(1, 129, 1)
    good_tes=np.delete(allTES, np.array([4,36,68,100])-1, axis=0)
    k=0
    for j in [1, 2]:
        for i in good_tes:
            
            #print(i)
            xtes, ytes, FP_index, index_q= scal.TES_Instru2coord(TES=i, ASIC=j, q=q, frame='ONAFP', verbose=False)
            ind=np.where((np.round(xtes, 4) == np.round(X, 4)) & (np.round(ytes, 4) == np.round(Y, 4)))
            #print(ind)
            place_graph=ind[0][0]*17+ind[1][0]+1
            #print(place_graph)
            #pixok=mymaps[k]!= hp.UNSEEN
            #res=mymaps[k]/np.max(mymaps[k])
            #res[~pixok]=hp.UNSEEN
            mytes=i
            if j == 2:
                mytes+=128
            hp.gnomview(maps[mytes-1], rot=[-14, 50], reso=8, sub=(17, 17, place_graph), cmap='jet', #title='TES : {}'.format(i), 
                notext=True, cbar=False, title='', margins=(0.001, 0.001, 0.001, 0.001))
            
            annotate('TES = {}'.format(mytes), xy=(0, 0),  xycoords='axes fraction', fontsize=15, color='black', 
                     fontstyle='italic', xytext=(0.2, 0.8))
            bar.update()
            if add_moon_traj is not None:
                hp.projplot(th, phi, color='k', lonlat=False, alpha=0.8, lw=2)
            
            
            k+=1
    if savepdf != None:
        savefig(savepdf, format="pdf", bbox_inches="tight")
    show()
    
# Get a dictionary
basedir = Qubic_DataDir()
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)

# If there is not this command, the kernel shut down every time..
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
day = '2022-07-14'
#day= '2020-11-10'
keyword = '**'
#keyword= '*test'
#data_dir = '/sps/hep/qubic/Data/'+day+'/'
data_dir = day+'/'
dirs = np.sort(glob.glob(data_dir+keyword))
print(dirs)

In [ ]:
#day= '2020-11-10'
keyword = '**'
#keyword= '*test'
#data_dir = '/sps/hep/qubic/Data/'+day+'/'
data_dir = day+'/'
dirs = np.sort(glob.glob(data_dir+keyword))
print(dirs)

ifile = 0
thedir = dirs[ifile]
print(thedir)
#note here is how you load the data in memory!
a = qubicfp()
a.read_qubicstudio_dataset(thedir)
#a.read_qubicstudio_dataset('/path/to/dataset')
az = a.azimuth()
el = a.elevation()
thk = a.timeaxis(datatype='hk')


In [ ]:
print(a.hk.keys())
a.hk.keys()

print("The keys in this dictionary are:\n")
for k in a.hk['CALSOURCE-CONF'].keys():
    print(k, a.hk['CALSOURCE-CONF'][k])

In [ ]:
from importlib import reload
reload(DH)

# Construction of the object
analysis=DH.BeamMapsAnalysis(a)

In [ ]:
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, BaseRADecFrame, get_moon, get_sun

Salta = EarthLocation(lat=-24.731358*u.deg, lon=-65.409535*u.deg, height=1152*u.m)
utcoffset = -3*u.hour  # Eastern Daylight Time

door_az = 116.4
az_span = 30.
el_min = 30.
el_max = 50.

day='2022-07-14'
start_obs_hour = '00:00:00'
date = Time(day+ ' 00:00:00')
start_obs_date = day +' '+start_obs_hour
delta_time = np.linspace(12,30,1000)*u.hour
time0 = Time(start_obs_date)-utcoffset
alltimes = time0 + delta_time
local_time_hours = ((Time(start_obs_date) + delta_time).cxcsec - date.cxcsec)/3600
### Local coordinates
frame_Salta = AltAz(obstime=alltimes, location=Salta)
### Moon
moon_Salta = get_moon(alltimes)
moonaltazs_Salta = moon_Salta.transform_to(frame_Salta)
delta_el = 20
valid = (moonaltazs_Salta.alt.value < (el_max+delta_el)) & (moonaltazs_Salta.alt.value > (el_min-delta_el)) & (moonaltazs_Salta.az.value > 90) & (moonaltazs_Salta.az.value < 120)
tstart = np.min(local_time_hours[valid])
tstop = np.max(local_time_hours[valid])
local_start = str(Time(start_obs_date)+tstart*u.hour)[:16]
local_stop = str(Time(start_obs_date)+tstop*u.hour)[:16]
UTC_start = str(Time(start_obs_date)-utcoffset+tstart*u.hour)[:16]
UTC_stop = str(Time(start_obs_date)-utcoffset+tstop*u.hour)[:16]

# Now let's prepare a back & forth in elevation scanning strategy as provided in qubic_soft
d = qubic.qubicdict.qubicDict()
d.read_from_file('MoonObservation.dict')
d['date_obs'] = str(Time(start_obs_date)-utcoffset+tstart*u.hour)
d['latitude'] = -24.731377    ### Salta Regional Noroeste
d['longitude'] = -65.409546   ### Salta Regional Noroeste
d['sampling'] = 1.
moon_ra_mean = np.mean(moon_Salta.ra[valid]/u.deg)
moon_dec_mean = np.mean(moon_Salta.dec[valid]/u.deg)
d['RA_center'] = moon_ra_mean #deg
d['DEC_center'] = moon_dec_mean #deg
d['duration'] = tstop-tstart # Hours
d['angspeed'] = 0.8 #deg/s
d['delta_az'] = az_span #deg
d['nsweeps_per_elevation'] = 1
d['angspeed_psi'] = 0. #deg/s
d['dead_time']=74.05
backforthdt = d['delta_az'] / d['angspeed'] * 2
print('Scan Duration: ',backforthdt)
print('Dead Time = {}'.format(d['dead_time']))




n_elevations = int(d['duration']*3600/(backforthdt+d['dead_time']))+1
el_step = np.round((el_max - el_min) / n_elevations * 100) / 100
d['fix_azimuth'] = {'apply':True,'az':105.,
                         'el':40,'el_step':el_step, 'fix_hwp':True, 'fix_pitch':True}
print(d['fix_azimuth'])
print('call')
p = qubic.get_pointing(d)

def give_me_moon_traj(az, el):
    th=np.zeros(len(az))
    phi=np.zeros(len(az))
    for i in range(len(az)):
        ips = hp.ang2pix(256, az[i]-120, el[i], lonlat=True)
        th[i], phi[i] = hp.pix2ang(256, ips)
    return th, phi

def do_cut_in_elevation(maps, elcut):
    if elcut is not None:
        el_min=elcut[0]
        el_max=elcut[1]
    
    index=np.where(maps!=hp.UNSEEN)[0]
    
    for ii, i in enumerate(index):
        az, el = hp.pix2ang(256, i, lonlat=True)
        #print(az-360, el)

        if elcut is not None:
            if el < el_min or el > el_max :
                maps[i]=hp.UNSEEN
        
    return maps

def projplot_healpy(az, el, diff_az, diff_el, c, lw, alpha, ls='-'):
    hp.projplot(az-diff_az, el-diff_el, 
            color=c, lonlat=True, alpha=alpha, lw=lw, ls=ls)
    
def make_healpix_map(azt, elt, tod, countcut=0):
    nside=256
    unseen_val=hp.UNSEEN
    ips = hp.ang2pix(nside, azt, elt, lonlat=True)
    mymap = np.zeros(12*nside**2)
    mapcount = np.zeros(12*nside**2)
    for i in range(len(azt)):
        mymap[ips[i]] -= tod[i]
        mapcount[ips[i]] += 1
    unseen = mapcount <= countcut
    mymap[unseen] = unseen_val
    mapcount[unseen] = unseen_val
    mymap[~unseen] = mymap[~unseen] / mapcount[~unseen]
    return mymap

# Raw map

In [ ]:
TESNum1=57 #40  #57 #46 #69[4e-2, 10]
mymaps1=analysis.fullanalysis(number_of_tes=TESNum1, filter=None, demod=False, remove_noise=True, 
                             make_maps='healpix', doplot=False, save=False)

TESNum2=46 #40  #57 #46 #69[4e-2, 10]
mymaps2=analysis.fullanalysis(number_of_tes=TESNum2, filter=None, demod=False, remove_noise=True, 
                             make_maps='healpix', doplot=False, save=False)

import pickle
with open('pointing_offsets.pickle', 'rb') as f:
    data = pickle.load(f)
#data[:, 0]+=0.9

In [ ]:
from qubic import fibtools as ft
import scipy.ndimage.filters as f

scantype_hk, azt, elt, scantype=identify_scans(thk, az, el, tt=analysis.tt, median_size=101, 
                                               thr_speedmin=0.1, doplot=False, plotrange=[0,1000])

In [ ]:
mytod1=ft.filter_data(analysis.tt, analysis.tod[TESNum1-1], lowcut=9e-3, highcut=2e-0, order=3)

%matplotlib notebook
plt.figure(figsize=(15, 5))
spectrum_f, freq_f = ft.power_spectrum(analysis.tt, mytod1, rebin=True)
spectrum_f2, freq_f2 = ft.power_spectrum(analysis.tt, analysis.tod[TESNum1-1], rebin=True)
plt.plot(freq_f, f.gaussian_filter1d(spectrum_f,1), label='Raw Data')
plt.plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1), label='Filtered data')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectrum')
plt.show()

az_point=116.4
delta_az=30
az_real=120
delta_pointing=az_real-az_point

In [ ]:
%matplotlib notebook
plt.figure(figsize=(15, 5))
plt.plot(analysis.tt, mytod1)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(20, 20))

lowel=40
highel=50

my1=make_healpix_map(azt-3.6, elt, mytod1)
my1=do_cut_in_elevation(my1, elcut=[lowel, highel])

my2=make_healpix_map(azt-3.6, elt, mytod2)
my2=do_cut_in_elevation(my2, elcut=[lowel, highel])


hp.gnomview(my1, rot=[-17, 42], reso=8, cmap='jet')
plt.title('TES : {}'.format(TESNum1))

# Do edge
#projplot_healpy(-az_real+np.ones(100)*(az_point-delta_az), np.linspace(lowel, highel, 100), 0, 0, c='red', lw=2, alpha=0.8)
#projplot_healpy(-az_real+np.ones(100)*az_point, np.linspace(lowel, highel, 100), 0, 0, c='red', lw=2, alpha=0.8)
#
#projplot_healpy(-az_real+np.linspace(az_point-delta_az, az_point, 100), 
#                np.ones(100)*lowel, 0, 0, c='red', lw=2, alpha=0.8)
#projplot_healpy(-az_real+np.linspace(az_point-delta_az, az_point, 100), 
#                np.ones(100)*highel, 0, 0, c='red', lw=2, alpha=0.8)

# Annotate

#az_min, az_max=86, 116
#for ii, i in enumerate(np.linspace(lowel, highel, 10)):
    #print(i)
#    projplot_healpy(-az_real+np.linspace(az_min, az_max, 100), np.ones(100)*i, 0, 0, c='black', lw=1, alpha=0.5)
    
#for ii, i in enumerate(np.linspace(az_min, az_max, 20)):
    #print(i)
#    projplot_healpy(-az_real+np.ones(100)*(i), np.linspace(lowel, highel, 100), 0, 0, c='black', lw=1, alpha=0.5)

#plt.annotate('Elevation = {:.0f}°'.format(highel), xy=(0, 0),  xycoords='figure fraction', rotation=-5, 
#             xytext=(0.25, 0.60), textcoords='axes fraction', color='black', fontsize=10, fontstyle='italic')
#plt.annotate('Elevation = {:.0f}°'.format(lowel), xy=(0, 0),  xycoords='figure fraction', rotation=-3, 
#             xytext=(0.2, 0.25), textcoords='axes fraction', color='black', fontsize=10, fontstyle='italic')
#plt.annotate('Azimuth = {:.1f}° (0°)'.format(az_point), xy=(0, 0),  xycoords='figure fraction', rotation=-80, 
#             xytext=(0.87, 0.32), textcoords='axes fraction', color='black', fontsize=10, fontstyle='italic')
#plt.annotate('Azimuth = {:.1f}° (-30°)'.format(az_point-30), xy=(0, 0),  xycoords='figure fraction', rotation=80, 
#             xytext=(0.12, 0.33), textcoords='axes fraction', color='black', fontsize=10, fontstyle='italic')

plt.show()

# Let's see the effect of the order

In [ ]:
lc=0.005
hc=1

tab_order=[0, 1, 2, 5]#np.arange(0, 6, 1)
tab_tod=np.zeros((len(tab_order), 1998848))
for ii, i in enumerate(tab_order):
    mytod=ft.filter_data(analysis.tt, analysis.tod[TESNum1-1], lowcut=lc, highcut=hc, order=i)
    tab_tod[ii]=mytod.copy()

%matplotlib inline
plt.figure(figsize=(15, 5))
spectrum_f2, freq_f2 = ft.power_spectrum(analysis.tt, analysis.tod[TESNum1-1], rebin=True)
plt.plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1), '--b', label='Raw data')
for i in range(len(tab_order)):
    spectrum_f, freq_f = ft.power_spectrum(analysis.tt, tab_tod[i], rebin=True)
    plt.plot(freq_f, f.gaussian_filter1d(spectrum_f,1), label='Filtered Data')
    
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectrum')

#plt.axvline(lc, color='black', lw=5)
#plt.axvline(hc, color='black', lw=5)
plt.show()

In [ ]:
%matplotlib notebook
plt.figure(figsize=(15, 5))
for i in range(len(tab_order)):
    plt.plot(analysis.tt, tab_tod[i])
plt.show()

In [ ]:
%matplotlib inline

plt.figure(figsize=(10, 10))

lowel=40
highel=45

for i in range(len(tab_order)):
    my1=make_healpix_map(azt, elt, tab_tod[i])
    my1=do_cut_in_elevation(my1, elcut=[lowel, highel])


    hp.gnomview(my1, rot=[-16, 42], reso=10, cmap='jet', sub=(1, 4, i+1))
    plt.title('TES : {} - Order = {}'.format(TESNum1, tab_order[i]))
plt.show()